# Hybrid QKAN on CIFAR-100

In this example, we demonstrate the performance and scalability of the **Hybrid QKAN (HQKAN)** architecture when applied to a large-scale model using the **CIFAR-100** dataset.

## Motivation

Due to the architectural design of Kolmogorov–Arnold Networks (KAN), the number of parameters grows **quadratically** with respect to the number of input and output channels. This presents a scalability challenge for large models.

Here is how the size increases:

In [1]:
from qkan import QKAN

model_1 = QKAN([10, 10])
model_2 = QKAN([10, 100])

print("Model 1 with shape [10, 10] has # of parameters:", model_1.param_size)
print("Model 2 with shape [10, 100] has # of parameters:", model_2.param_size)
del model_1, model_2

Model 1 with shape [10, 10] has # of parameters: 800
Model 2 with shape [10, 100] has # of parameters: 8000


To address this, we propose a **hybrid architecture** that integrates two fully connected networks (FCNs) to **compress** and **reconstruct** the feature representation. This forms an **autoencoder-like bottleneck structure** around the QKAN core.

We refer to this approach as **Hybrid QKAN (HQKAN)**.

## Architecture Overview

- **Encoder FCN** compresses the high-dimensional input features.
- **QKAN Core** processes the reduced latent representation.
- **Decoder FCN** reconstructs the output to the original dimensionality.
- This structure allows us to benefit from QKAN's expressiveness while controlling parameter growth.

## Experiment: CIFAR-100

We apply HQKAN to the CIFAR-100 dataset to evaluate its performance and efficiency.

In [2]:
import random
import sys

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
class CNet(nn.Module):
    def __init__(self, device):
        super(CNet, self).__init__()

        self.device = device

        self.cnn1 = nn.Conv2d(
            in_channels=3, out_channels=32, kernel_size=3, device=device
        )
        self.relu = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.cnn2 = nn.Conv2d(
            in_channels=32, out_channels=64, kernel_size=3, device=device
        )
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.cnn3 = nn.Conv2d(
            in_channels=64, out_channels=64, kernel_size=3, device=device
        )
        self.maxpool3 = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        x = x.to(self.device)
        x = self.cnn1(x)
        x = self.relu(x)
        x = self.maxpool1(x)
        x = self.cnn2(x)
        x = self.relu(x)
        x = self.maxpool2(x)
        x = self.cnn3(x)
        x = self.relu(x)
        x = self.maxpool3(x)
        x = x.flatten(start_dim=1)
        return x

In [4]:
# Load CIFAR100 dataset
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))]
)
trainset = torchvision.datasets.CIFAR100(
    root="./data", train=True, download=True, transform=transform
)
testset = torchvision.datasets.CIFAR100(
    root="./data", train=False, download=True, transform=transform
)
trainloader = DataLoader(trainset, batch_size=1000, shuffle=True)
testloader = DataLoader(testset, batch_size=1000, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
in_feat = 2 * 2 * 64
out_feat = 100
in_resize = np.ceil(np.log2(in_feat)).astype(int)
out_resize = np.ceil(np.log2(out_feat)).astype(int)
model = nn.Sequential(
    CNet(device=device),
    nn.Linear(in_feat, in_resize, device=device),
    QKAN([in_resize, out_resize], device=device),
    nn.Linear(out_resize, out_feat, device=device),
)

criterion = nn.CrossEntropyLoss()
accs = []
losses = []
test_accs = []
test_losses = []
test_top5_accs = []

optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [6]:
def train(model, optimizer):
    for epoch in range(50):
        # Train
        model.train()
        with tqdm(trainloader) as pbar:
            for i, (images, labels) in enumerate(pbar):
                optimizer.zero_grad()
                output = model(images)
                loss = criterion(output, labels.to(device))
                loss.backward()
                optimizer.step()
                accuracy = (output.argmax(dim=1) == labels.to(device)).float().mean()
                accs.append(accuracy.item())
                losses.append(loss.item())
                pbar.set_postfix(
                    loss=loss.item(),
                    accuracy=accuracy.item(),
                    lr=optimizer.param_groups[0]["lr"],
                )

        # test
        model.eval()
        test_loss = 0
        test_accuracy = 0
        test_top5_accuracy = 0
        with torch.no_grad():
            for images, labels in testloader:
                output = model(images)
                test_loss += criterion(output, labels.to(device)).item()
                test_accuracy += (
                    (output.argmax(dim=1) == labels.to(device)).float().mean().item()
                )
                test_top5_accuracy += (
                    (output.topk(5, dim=1).indices == labels.to(device).unsqueeze(1)).any(dim=1).float().mean().item()
                )
        test_loss /= len(testloader)
        test_accuracy /= len(testloader)
        test_top5_accuracy /= len(testloader)

        print(f"Epoch {epoch + 1}, test Loss: {test_loss}, test Accuracy: {test_accuracy}")
        test_accs.append(test_accuracy)
        test_losses.append(test_loss)
        test_top5_accs.append(test_top5_accuracy)

In [7]:
train(model, optimizer)

100%|██████████| 50/50 [00:02<00:00, 23.03it/s, accuracy=0.014, loss=4.57, lr=0.001]


Epoch 1, test Loss: 4.5694972515106205, test Accuracy: 0.01120000034570694


100%|██████████| 50/50 [00:01<00:00, 25.58it/s, accuracy=0.032, loss=4.47, lr=0.001]


Epoch 2, test Loss: 4.473529720306397, test Accuracy: 0.02420000098645687


100%|██████████| 50/50 [00:01<00:00, 25.93it/s, accuracy=0.04, loss=4.35, lr=0.001] 


...

Epoch 48, test Loss: 3.1972729444503782, test Accuracy: 0.20940001010894777


100%|██████████| 50/50 [00:01<00:00, 25.62it/s, accuracy=0.255, loss=2.86, lr=0.001]


Epoch 49, test Loss: 3.210345673561096, test Accuracy: 0.20560001134872435


100%|██████████| 50/50 [00:02<00:00, 24.92it/s, accuracy=0.262, loss=2.77, lr=0.001]


Epoch 50, test Loss: 3.1845874547958375, test Accuracy: 0.21320001184940338


In [8]:
print("Top-1 Accuracy: ", max(test_accs))
print("Top-5 Accuracy: ", max(test_top5_accs))

print("CNN+HQKAN Model size:", param:=len(torch.concat([param.cpu().detach().flatten() for param in model.parameters() if param.requires_grad])))
cnn = CNet(device="cpu")
cnn_param = len(torch.concat([param.cpu().detach().flatten() for param in cnn.parameters() if param.requires_grad]))
del cnn
print("CNN part size:", cnn_param)
print("HQKAN part size:", param - cnn_param)

Top-1 Accuracy:  0.21320001184940338
Top-5 Accuracy:  0.5048000246286393
CNN+HQKAN Model size: 59624
CNN part size: 56320
HQKAN part size: 3304


To enhance model capacity, we increase the input and output channels of the QKAN core. This enables learning of more complex feature representations while maintaining the parameter budget via the hybrid bottleneck design.

In [9]:
in_resize = np.ceil(np.log2(in_feat)).astype(int) * 4
out_resize = np.ceil(np.log2(out_feat)).astype(int) * 4
model = nn.Sequential(
    CNet(device=device),
    nn.Linear(in_feat, in_resize, device=device),
    QKAN([in_resize, out_resize], device=device),
    nn.Linear(out_resize, out_feat, device=device),
)

criterion = nn.CrossEntropyLoss()
accs = []
losses = []
test_accs = []
test_losses = []
test_top5_accs = []

optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [10]:
train(model, optimizer)

100%|██████████| 50/50 [00:02<00:00, 23.65it/s, accuracy=0.04, loss=4.37, lr=0.001] 


Epoch 1, test Loss: 4.359431600570678, test Accuracy: 0.04140000194311142


100%|██████████| 50/50 [00:02<00:00, 23.53it/s, accuracy=0.055, loss=4.03, lr=0.001]


Epoch 2, test Loss: 4.007006192207337, test Accuracy: 0.08220000267028808


100%|██████████| 50/50 [00:02<00:00, 23.57it/s, accuracy=0.107, loss=3.87, lr=0.001]


...

Epoch 48, test Loss: 2.555943727493286, test Accuracy: 0.34780001938343047


100%|██████████| 50/50 [00:02<00:00, 23.55it/s, accuracy=0.454, loss=2.12, lr=0.001]


Epoch 49, test Loss: 2.524978685379028, test Accuracy: 0.36210001111030576


100%|██████████| 50/50 [00:02<00:00, 22.87it/s, accuracy=0.437, loss=2.07, lr=0.001]


Epoch 50, test Loss: 2.5304693937301637, test Accuracy: 0.35640001893043516


In [11]:
print("Top-1 Accuracy: ", max(test_accs))
print("Top-5 Accuracy: ", max(test_top5_accs))

print("CNN+HQKAN-44 Model size:", param:=len(torch.concat([param.cpu().detach().flatten() for param in model.parameters() if param.requires_grad])))
print("CNN part size:", cnn_param)
print("HQKAN-44 part size:", param - cnn_param)

Top-1 Accuracy:  0.36210001111030576
Top-5 Accuracy:  0.6717000305652618
CNN+HQKAN-44 Model size: 74612
CNN part size: 56320
HQKAN-44 part size: 18292


Compare to MLP

In [12]:
class CNN(nn.Module):
    def __init__(self, device):
        super(CNN, self).__init__()
        self.cnet = CNet(device)
        self.fc1 = nn.Linear(256, 192).to(device)
        self.fc2 = nn.Linear(192, 128).to(device)
        self.fc3 = nn.Linear(128, 100).to(device)
        self.device = device
    
    def forward(self, x):
        x.to(self.device)
        x = self.cnet(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        return x

model = CNN(device=device)

criterion = nn.CrossEntropyLoss()
accs = []
losses = []
test_accs = []
test_losses = []
test_top5_accs = []

optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [13]:
train(model, optimizer)

100%|██████████| 50/50 [00:01<00:00, 26.49it/s, accuracy=0.073, loss=4.02, lr=0.001]


Epoch 1, test Loss: 4.0597692966461185, test Accuracy: 0.07460000440478325


100%|██████████| 50/50 [00:01<00:00, 26.60it/s, accuracy=0.085, loss=3.85, lr=0.001]


Epoch 2, test Loss: 3.8250439167022705, test Accuracy: 0.10800000503659249


100%|██████████| 50/50 [00:01<00:00, 26.46it/s, accuracy=0.127, loss=3.7, lr=0.001] 


...

Epoch 48, test Loss: 2.5316006898880006, test Accuracy: 0.37280001640319826


100%|██████████| 50/50 [00:01<00:00, 26.48it/s, accuracy=0.473, loss=1.96, lr=0.001]


Epoch 49, test Loss: 2.561800479888916, test Accuracy: 0.3666000127792358


100%|██████████| 50/50 [00:01<00:00, 26.64it/s, accuracy=0.461, loss=1.99, lr=0.001]


Epoch 50, test Loss: 2.515561318397522, test Accuracy: 0.3707000195980072


In [14]:
print("Top-1 Accuracy: ", max(test_accs))
print("Top-5 Accuracy: ", max(test_top5_accs))

print("CNN+MLP Model size:", param:=len(torch.concat([param.cpu().detach().flatten() for param in model.parameters() if param.requires_grad])))
print("CNN part size:", cnn_param)
print("MLP part size:", param - cnn_param)

Top-1 Accuracy:  0.3738000154495239
Top-5 Accuracy:  0.6787000298500061
CNN+MLP Model size: 143268
CNN part size: 56320
MLP part size: 86948


## Results

- **HQKAN-44** achieves **comparable accuracy** to an MLP baseline.
- It uses only **20% of the parameters** compared to the MLP.
- Training time is also **comparable to MLP**, and **significantly faster** than many other quantum-inspired machine learning (QML) models when run on classical simulators.

## Summary

Hybrid QKAN offers a scalable and efficient alternative to traditional MLPs and other QML models:
- ✔ Comparable accuracy
- ✔ 5× fewer parameters
- ✔ Faster training than typical QML approaches

This demonstrates HQKAN's potential as a practical and effective model architecture for modern deep learning tasks.